## Prepare the environment

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [22]:
import os, sys
cur_dir = os.getcwd()
DATA_HOME_DIR = cur_dir + '/data'
DATA_HOME_DIR

'/home/ubuntu/nbs/statefarm/data'

In [23]:
sys.path.insert(1, os.path.join(sys.path[0], '../../utils'))

In [24]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

In [25]:
TRAIN_DATA_DIR = DATA_HOME_DIR + '/train'
VALID_DATA_DIR = DATA_HOME_DIR + '/valid'
TEST_DATA_DIR = DATA_HOME_DIR + '/test'

SAMPLE_TRAIN_DATA_DIR = DATA_HOME_DIR + '/sample/train'
SAMPLE_VALID_DATA_DIR = DATA_HOME_DIR + '/sample/valid'
SAMPLE_TEST_DATA_DIR = DATA_HOME_DIR + '/sample/test'

In [28]:
%cd $DATA_HOME_DIR

os.mkdir('valid')
os.mkdir('result')
os.mkdir('sample')
os.mkdir('sample/train')
os.mkdir('sample/valid')
os.mkdir('sample/test')
os.mkdir('sample/result')

/home/ubuntu/nbs/statefarm/data


In [29]:
DATA_HOME_DIR

'/home/ubuntu/nbs/statefarm/data'

In [42]:
%cd $TRAIN_DATA_DIR

/home/ubuntu/nbs/statefarm/data/train


In [32]:
for i in glob('c?'):
    os.mkdir('../sample/train/' + i)
    os.mkdir('../sample/valid/' + i)

In [43]:
for i in glob('c?'):
    os.mkdir('../valid/' + i)

In [44]:
for d in glob('c?'):
    g = glob(d+'/*.jpg')
    shuffle = np.random.permutation(g)
    for i in range(500):
        os.rename(shuffle[i], '../valid/' + shuffle[i])

## Copy files to sample data

In [33]:
from shutil import copyfile

In [37]:
os.getcwd()

'/home/ubuntu/nbs/statefarm/data/train'

In [38]:
for d in glob('c?'):
    g = glob(d+'/*.jpg')
    shuffle = np.random.permutation(g)
    for i in range(100):
        copyfile(shuffle[i], '../sample/train/' + shuffle[i])

In [45]:
%cd $VALID_DATA_DIR

/home/ubuntu/nbs/statefarm/data/valid


In [47]:
for d in glob('c?'):
    g = glob(d+'/*.jpg')
    shuffle = np.random.permutation(g)
    for i in range(25):
        copyfile(shuffle[i], '../sample/valid/' + shuffle[i])

## Model creation and fitting

### Linear model

In [49]:
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(SAMPLE_TRAIN_DATA_DIR, batch_size=64)
valid_batches = gen.flow_from_directory(SAMPLE_VALID_DATA_DIR, batch_size=64)

Found 1000 images belonging to 10 classes.
Found 250 images belonging to 10 classes.


In [63]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3,256,256)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_3 (BatchNormal(None, 3, 256, 256)   6           batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 196608)        0           batchnormalization_3[0][0]       
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 10)            1966090     flatten_3[0][0]                  
Total params: 1966096
____________________________________________________________________________________________________


In [65]:
# modify learning rate to see improvement
model.optimizer.lr = 1e-4
model.fit_generator(batches, batches.N, nb_epoch=3,
                    validation_data=valid_batches, nb_val_samples=valid_batches.N)

Epoch 1/3
1000/1000 [==============================] - 20s - loss: 1.2787 - acc: 0.6210 - val_loss: 1.4516 - val_acc: 0.5520
Epoch 2/3
1000/1000 [==============================] - 19s - loss: 1.0539 - acc: 0.7290 - val_loss: 1.5118 - val_acc: 0.5640
Epoch 3/3
1000/1000 [==============================] - 15s - loss: 0.8986 - acc: 0.7920 - val_loss: 1.2096 - val_acc: 0.5960


In [66]:
np.round(model.predict_generator(batches, batches.N)[:10],2)

array([[ 0.6 ,  0.03,  0.01,  0.01,  0.09,  0.05,  0.  ,  0.  ,  0.  ,  0.21],
       [ 0.  ,  0.03,  0.  ,  0.01,  0.06,  0.01,  0.  ,  0.03,  0.05,  0.81],
       [ 0.71,  0.03,  0.09,  0.  ,  0.11,  0.01,  0.  ,  0.  ,  0.02,  0.02],
       [ 0.01,  0.  ,  0.01,  0.04,  0.64,  0.  ,  0.01,  0.01,  0.  ,  0.28],
       [ 0.02,  0.01,  0.  ,  0.09,  0.75,  0.  ,  0.  ,  0.04,  0.  ,  0.07],
       [ 0.  ,  0.  ,  0.  ,  0.22,  0.44,  0.01,  0.  ,  0.  ,  0.01,  0.3 ],
       [ 0.  ,  0.04,  0.37,  0.07,  0.1 ,  0.01,  0.01,  0.13,  0.04,  0.23],
       [ 0.01,  0.84,  0.05,  0.01,  0.01,  0.01,  0.01,  0.01,  0.02,  0.04],
       [ 0.  ,  0.  ,  0.06,  0.  ,  0.02,  0.83,  0.04,  0.01,  0.01,  0.03],
       [ 0.6 ,  0.01,  0.  ,  0.01,  0.11,  0.  ,  0.01,  0.  ,  0.01,  0.25]], dtype=float32)

### Single hidden layer (linear model)

In [67]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3,256,256)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_4 (BatchNormal(None, 3, 256, 256)   6           batchnormalization_input_4[0][0] 
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 196608)        0           batchnormalization_4[0][0]       
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 100)           19660900    flatten_4[0][0]                  
____________________________________________________________________________________________________
batchnormalization_5 (BatchNormal(None, 100)           200         dense_4[0][0]                    
___________________________________________________________________________________________

In [72]:
# modify learning rate to see improvement
model.optimizer.lr = 1e-5
model.fit_generator(batches, batches.N, nb_epoch=3,
                    validation_data=valid_batches, nb_val_samples=valid_batches.N)

Epoch 1/3
1000/1000 [==============================] - 19s - loss: 0.2680 - acc: 0.9860 - val_loss: 1.1185 - val_acc: 0.6920
Epoch 2/3
1000/1000 [==============================] - 15s - loss: 0.1777 - acc: 0.9960 - val_loss: 0.8746 - val_acc: 0.7840
Epoch 3/3
1000/1000 [==============================] - 15s - loss: 0.1291 - acc: 0.9980 - val_loss: 0.7733 - val_acc: 0.7560


### Simple Conv layer

In [83]:
def get_conv_model():
    model = Sequential()
    model.add(BatchNormalization(axis=1, input_shape=(3,256,256)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64,3,3,activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [82]:
# modify learning rate to see improvement
model = get_conv_model()
model.optimizer.lr = 1e-5
model.fit_generator(batches, batches.N, nb_epoch=3,
                    validation_data=valid_batches, nb_val_samples=valid_batches.N)

Epoch 1/3
1000/1000 [==============================] - 25s - loss: 0.0248 - acc: 1.0000 - val_loss: 1.3129 - val_acc: 0.7080
Epoch 2/3
1000/1000 [==============================] - 23s - loss: 0.0213 - acc: 1.0000 - val_loss: 1.1785 - val_acc: 0.7800
Epoch 3/3
1000/1000 [==============================] - 22s - loss: 0.0180 - acc: 1.0000 - val_loss: 1.1804 - val_acc: 0.7640


### Data augmentation

In [84]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = gen.flow_from_directory(SAMPLE_TRAIN_DATA_DIR, batch_size=64)

Found 1000 images belonging to 10 classes.


In [85]:
model = get_conv_model()

In [88]:
# modify learning rate to see improvement
model.optimizer.lr = 1e-4
model.fit_generator(batches, batches.N, nb_epoch=3,
                    validation_data=valid_batches, nb_val_samples=valid_batches.N)

Epoch 1/3
1000/1000 [==============================] - 30s - loss: 0.0266 - acc: 1.0000 - val_loss: 1.1211 - val_acc: 0.7880
Epoch 2/3
1000/1000 [==============================] - 24s - loss: 0.0228 - acc: 1.0000 - val_loss: 1.1866 - val_acc: 0.7680
Epoch 3/3
1000/1000 [==============================] - 22s - loss: 0.0195 - acc: 1.0000 - val_loss: 1.1274 - val_acc: 0.7920
